In [64]:
import torch
from torch import nn
from torchvision import datasets, transforms
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [30]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),  # mean value = 0.1307, standard deviation value = 0.3081
])


In [31]:
data_path = './MNIST'

training_set = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)
testing_set = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)


In [45]:
training_set[0][0]
training_set[0][1]
# training_set is composed of image and label

5

In [46]:
type(training_set[0][0])
#already tensor, no need to convert

torch.Tensor

In [33]:
class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        
        # construct layers for a neural network
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=20*20),
            nn.Sigmoid(),
        ) 
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=20*20, out_features=10*10),
            nn.Sigmoid(),
        ) 
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=10*10, out_features=10),
            nn.LogSoftmax(dim=1),
        ) 
        
        
    def forward(self, inputs):                 # [batchSize, 1, 28, 28]
        x = inputs.view(inputs.size(0), -1)    # [batchSize, 28*28]
        x = self.classifier1(x)                # [batchSize, 20*20]
        x = self.classifier2(x)                # [batchSize, 10*10]
        out = self.classifier3(x)              # [batchSize, 10]
        
        return out


In [34]:
criterion = nn.NLLLoss()

In [35]:
device="cpu"
learning_rate_value=0.01

In [36]:
classifier = classification().to(device)
optimizer = torch.optim.SGD(classifier.parameters(), lr=learning_rate_value)


In [62]:
#start training with batchsize 32
classifier = classification().to("cpu")
optimizer = torch.optim.SGD(classifier.parameters(), lr=learning_rate_value)
criterion = nn.NLLLoss()
batch_size=32
epochs = 30
loss_history=np.zeros(epochs)
accuracy_history=np.zeros(epochs)
batched_train = torch.utils.data.DataLoader(training_set, 32, shuffle=True)
batched_test = torch.utils.data.DataLoader(testing_set, 32, shuffle=True)
for i in range(epoch):
    total_loss = 0
    
    for img, label in batched_train:
        optimizer.zero_grad()
        predicted = classifier.forward(img)
        loss = criterion(predicted, label)
        loss.backward
        optimizer.step()
        lo
    for img, label in testingset:
    
    

tensor(2.3667, grad_fn=<NllLossBackward>)
tensor(2.3261, grad_fn=<NllLossBackward>)
tensor(2.2260, grad_fn=<NllLossBackward>)
tensor(2.2740, grad_fn=<NllLossBackward>)
tensor(2.3357, grad_fn=<NllLossBackward>)
tensor(2.3389, grad_fn=<NllLossBackward>)
tensor(2.3636, grad_fn=<NllLossBackward>)
tensor(2.4366, grad_fn=<NllLossBackward>)
tensor(2.2704, grad_fn=<NllLossBackward>)
tensor(2.4272, grad_fn=<NllLossBackward>)
tensor(2.3688, grad_fn=<NllLossBackward>)
tensor(2.3935, grad_fn=<NllLossBackward>)
tensor(2.4030, grad_fn=<NllLossBackward>)
tensor(2.3246, grad_fn=<NllLossBackward>)
tensor(2.3192, grad_fn=<NllLossBackward>)
tensor(2.3925, grad_fn=<NllLossBackward>)
tensor(2.2595, grad_fn=<NllLossBackward>)
tensor(2.2679, grad_fn=<NllLossBackward>)
tensor(2.3609, grad_fn=<NllLossBackward>)
tensor(2.3230, grad_fn=<NllLossBackward>)
tensor(2.2281, grad_fn=<NllLossBackward>)
tensor(2.3544, grad_fn=<NllLossBackward>)
tensor(2.3769, grad_fn=<NllLossBackward>)
tensor(2.3883, grad_fn=<NllLossBac

tensor(2.3908, grad_fn=<NllLossBackward>)
tensor(2.2914, grad_fn=<NllLossBackward>)
tensor(2.2658, grad_fn=<NllLossBackward>)
tensor(2.3279, grad_fn=<NllLossBackward>)
tensor(2.3561, grad_fn=<NllLossBackward>)
tensor(2.4122, grad_fn=<NllLossBackward>)
tensor(2.3278, grad_fn=<NllLossBackward>)
tensor(2.3049, grad_fn=<NllLossBackward>)
tensor(2.3848, grad_fn=<NllLossBackward>)
tensor(2.3764, grad_fn=<NllLossBackward>)
tensor(2.2935, grad_fn=<NllLossBackward>)
tensor(2.3182, grad_fn=<NllLossBackward>)
tensor(2.2887, grad_fn=<NllLossBackward>)
tensor(2.3583, grad_fn=<NllLossBackward>)
tensor(2.4215, grad_fn=<NllLossBackward>)
tensor(2.3333, grad_fn=<NllLossBackward>)
tensor(2.3446, grad_fn=<NllLossBackward>)
tensor(2.2768, grad_fn=<NllLossBackward>)
tensor(2.2973, grad_fn=<NllLossBackward>)
tensor(2.3244, grad_fn=<NllLossBackward>)
tensor(2.3975, grad_fn=<NllLossBackward>)
tensor(2.4352, grad_fn=<NllLossBackward>)
tensor(2.3448, grad_fn=<NllLossBackward>)
tensor(2.3076, grad_fn=<NllLossBac

tensor(2.3220, grad_fn=<NllLossBackward>)
tensor(2.3171, grad_fn=<NllLossBackward>)
tensor(2.3065, grad_fn=<NllLossBackward>)
tensor(2.3543, grad_fn=<NllLossBackward>)
tensor(2.3377, grad_fn=<NllLossBackward>)
tensor(2.3394, grad_fn=<NllLossBackward>)
tensor(2.3752, grad_fn=<NllLossBackward>)
tensor(2.3237, grad_fn=<NllLossBackward>)
tensor(2.3251, grad_fn=<NllLossBackward>)
tensor(2.3803, grad_fn=<NllLossBackward>)
tensor(2.4326, grad_fn=<NllLossBackward>)
tensor(2.3128, grad_fn=<NllLossBackward>)
tensor(2.3985, grad_fn=<NllLossBackward>)
tensor(2.2688, grad_fn=<NllLossBackward>)
tensor(2.4021, grad_fn=<NllLossBackward>)
tensor(2.3675, grad_fn=<NllLossBackward>)
tensor(2.4273, grad_fn=<NllLossBackward>)
tensor(2.3328, grad_fn=<NllLossBackward>)
tensor(2.2565, grad_fn=<NllLossBackward>)
tensor(2.3627, grad_fn=<NllLossBackward>)
tensor(2.4076, grad_fn=<NllLossBackward>)
tensor(2.4992, grad_fn=<NllLossBackward>)
tensor(2.3671, grad_fn=<NllLossBackward>)
tensor(2.4047, grad_fn=<NllLossBac

tensor(2.2949, grad_fn=<NllLossBackward>)
tensor(2.3769, grad_fn=<NllLossBackward>)
tensor(2.4287, grad_fn=<NllLossBackward>)
tensor(2.3398, grad_fn=<NllLossBackward>)
tensor(2.4344, grad_fn=<NllLossBackward>)
tensor(2.3279, grad_fn=<NllLossBackward>)
tensor(2.3169, grad_fn=<NllLossBackward>)
tensor(2.4655, grad_fn=<NllLossBackward>)
tensor(2.3771, grad_fn=<NllLossBackward>)
tensor(2.3007, grad_fn=<NllLossBackward>)
tensor(2.2821, grad_fn=<NllLossBackward>)
tensor(2.4088, grad_fn=<NllLossBackward>)
tensor(2.3090, grad_fn=<NllLossBackward>)
tensor(2.3593, grad_fn=<NllLossBackward>)
tensor(2.3064, grad_fn=<NllLossBackward>)
tensor(2.4308, grad_fn=<NllLossBackward>)
tensor(2.4274, grad_fn=<NllLossBackward>)
tensor(2.3930, grad_fn=<NllLossBackward>)
tensor(2.3142, grad_fn=<NllLossBackward>)
tensor(2.3607, grad_fn=<NllLossBackward>)
tensor(2.3431, grad_fn=<NllLossBackward>)
tensor(2.3429, grad_fn=<NllLossBackward>)
tensor(2.2577, grad_fn=<NllLossBackward>)
tensor(2.3787, grad_fn=<NllLossBac

tensor(2.2773, grad_fn=<NllLossBackward>)
tensor(2.3655, grad_fn=<NllLossBackward>)
tensor(2.3187, grad_fn=<NllLossBackward>)
tensor(2.3452, grad_fn=<NllLossBackward>)
tensor(2.3394, grad_fn=<NllLossBackward>)
tensor(2.3540, grad_fn=<NllLossBackward>)
tensor(2.4511, grad_fn=<NllLossBackward>)
tensor(2.3201, grad_fn=<NllLossBackward>)
tensor(2.3332, grad_fn=<NllLossBackward>)
tensor(2.4113, grad_fn=<NllLossBackward>)
tensor(2.2509, grad_fn=<NllLossBackward>)
tensor(2.3535, grad_fn=<NllLossBackward>)
tensor(2.3021, grad_fn=<NllLossBackward>)
tensor(2.4098, grad_fn=<NllLossBackward>)
tensor(2.3390, grad_fn=<NllLossBackward>)
tensor(2.3632, grad_fn=<NllLossBackward>)
tensor(2.3046, grad_fn=<NllLossBackward>)
tensor(2.3821, grad_fn=<NllLossBackward>)
tensor(2.3388, grad_fn=<NllLossBackward>)
tensor(2.3236, grad_fn=<NllLossBackward>)
tensor(2.2965, grad_fn=<NllLossBackward>)
tensor(2.4066, grad_fn=<NllLossBackward>)
tensor(2.3658, grad_fn=<NllLossBackward>)
tensor(2.3859, grad_fn=<NllLossBac

tensor(2.3246, grad_fn=<NllLossBackward>)
tensor(2.2726, grad_fn=<NllLossBackward>)
tensor(2.3867, grad_fn=<NllLossBackward>)
tensor(2.3501, grad_fn=<NllLossBackward>)
tensor(2.4073, grad_fn=<NllLossBackward>)
tensor(2.3489, grad_fn=<NllLossBackward>)
tensor(2.2816, grad_fn=<NllLossBackward>)
tensor(2.2603, grad_fn=<NllLossBackward>)
tensor(2.3847, grad_fn=<NllLossBackward>)
tensor(2.3367, grad_fn=<NllLossBackward>)
tensor(2.3632, grad_fn=<NllLossBackward>)
tensor(2.3292, grad_fn=<NllLossBackward>)
tensor(2.4133, grad_fn=<NllLossBackward>)
tensor(2.5055, grad_fn=<NllLossBackward>)
tensor(2.3433, grad_fn=<NllLossBackward>)
tensor(2.3218, grad_fn=<NllLossBackward>)
tensor(2.3322, grad_fn=<NllLossBackward>)
tensor(2.3997, grad_fn=<NllLossBackward>)
tensor(2.3034, grad_fn=<NllLossBackward>)
tensor(2.3206, grad_fn=<NllLossBackward>)
tensor(2.3391, grad_fn=<NllLossBackward>)
tensor(2.3108, grad_fn=<NllLossBackward>)
tensor(2.3332, grad_fn=<NllLossBackward>)
tensor(2.3907, grad_fn=<NllLossBac

tensor(2.2660, grad_fn=<NllLossBackward>)
tensor(2.3852, grad_fn=<NllLossBackward>)
tensor(2.3767, grad_fn=<NllLossBackward>)
tensor(2.3456, grad_fn=<NllLossBackward>)
tensor(2.2737, grad_fn=<NllLossBackward>)
tensor(2.3361, grad_fn=<NllLossBackward>)
tensor(2.3507, grad_fn=<NllLossBackward>)
tensor(2.3751, grad_fn=<NllLossBackward>)
tensor(2.3394, grad_fn=<NllLossBackward>)
tensor(2.3805, grad_fn=<NllLossBackward>)
tensor(2.3457, grad_fn=<NllLossBackward>)
tensor(2.3863, grad_fn=<NllLossBackward>)
tensor(2.3317, grad_fn=<NllLossBackward>)
tensor(2.4026, grad_fn=<NllLossBackward>)
tensor(2.3126, grad_fn=<NllLossBackward>)
tensor(2.2958, grad_fn=<NllLossBackward>)
tensor(2.3087, grad_fn=<NllLossBackward>)
tensor(2.3751, grad_fn=<NllLossBackward>)
tensor(2.3643, grad_fn=<NllLossBackward>)
tensor(2.3603, grad_fn=<NllLossBackward>)
tensor(2.3539, grad_fn=<NllLossBackward>)
tensor(2.3743, grad_fn=<NllLossBackward>)
tensor(2.3594, grad_fn=<NllLossBackward>)
tensor(2.2872, grad_fn=<NllLossBac

tensor(2.3332, grad_fn=<NllLossBackward>)
tensor(2.2496, grad_fn=<NllLossBackward>)
tensor(2.3019, grad_fn=<NllLossBackward>)
tensor(2.3483, grad_fn=<NllLossBackward>)
tensor(2.2893, grad_fn=<NllLossBackward>)
tensor(2.3573, grad_fn=<NllLossBackward>)
tensor(2.3683, grad_fn=<NllLossBackward>)
tensor(2.4078, grad_fn=<NllLossBackward>)
tensor(2.3378, grad_fn=<NllLossBackward>)
tensor(2.3402, grad_fn=<NllLossBackward>)
tensor(2.3907, grad_fn=<NllLossBackward>)
tensor(2.4737, grad_fn=<NllLossBackward>)
tensor(2.3856, grad_fn=<NllLossBackward>)
tensor(2.4069, grad_fn=<NllLossBackward>)
tensor(2.3097, grad_fn=<NllLossBackward>)
tensor(2.3739, grad_fn=<NllLossBackward>)
tensor(2.3597, grad_fn=<NllLossBackward>)
tensor(2.3001, grad_fn=<NllLossBackward>)
tensor(2.3679, grad_fn=<NllLossBackward>)
tensor(2.2332, grad_fn=<NllLossBackward>)
tensor(2.2040, grad_fn=<NllLossBackward>)
tensor(2.4359, grad_fn=<NllLossBackward>)
tensor(2.3635, grad_fn=<NllLossBackward>)
tensor(2.3977, grad_fn=<NllLossBac

tensor(2.3756, grad_fn=<NllLossBackward>)
tensor(2.2968, grad_fn=<NllLossBackward>)
tensor(2.2572, grad_fn=<NllLossBackward>)
tensor(2.3194, grad_fn=<NllLossBackward>)
tensor(2.5161, grad_fn=<NllLossBackward>)
tensor(2.3654, grad_fn=<NllLossBackward>)
tensor(2.3346, grad_fn=<NllLossBackward>)
tensor(2.3073, grad_fn=<NllLossBackward>)
tensor(2.2889, grad_fn=<NllLossBackward>)
tensor(2.2646, grad_fn=<NllLossBackward>)
tensor(2.4159, grad_fn=<NllLossBackward>)
tensor(2.3825, grad_fn=<NllLossBackward>)
tensor(2.3659, grad_fn=<NllLossBackward>)
tensor(2.3014, grad_fn=<NllLossBackward>)
tensor(2.3567, grad_fn=<NllLossBackward>)
tensor(2.3848, grad_fn=<NllLossBackward>)
tensor(2.3094, grad_fn=<NllLossBackward>)
tensor(2.4218, grad_fn=<NllLossBackward>)
tensor(2.3588, grad_fn=<NllLossBackward>)
tensor(2.3423, grad_fn=<NllLossBackward>)
tensor(2.4456, grad_fn=<NllLossBackward>)
tensor(2.3588, grad_fn=<NllLossBackward>)
tensor(2.3859, grad_fn=<NllLossBackward>)
tensor(2.3342, grad_fn=<NllLossBac

TypeError: 'DataLoader' object is not subscriptable

In [57]:
train_data_loader = torch.utils.data.DataLoader(training_set, 32, shuffle=True)


In [61]:
example_mini_batch_img, example_mini_batch_label  = next(iter(train_data_loader))
print(example_mini_batch_img.shape)

torch.Size([32, 1, 28, 28])


In [ ]:
def run_epoch (model, train_data, test_data, optimizer, criterion):
    
    for img_i, label_i in train_data:

        img_i, label_i = img_i.to(device), label_i.to(device)

        optimizer.zero_grad()

        # Forward
        label_predicted = mlp.forward(img_i.view(-1, 28*28))
        # Loss computation
        loss  = criterion(torch.log(label_predicted), label_i.view(-1))
        # Backward
        loss.backward()
        # Optimize for img_i
        optimizer.step()
    
    total_test_loss = 0
    for img_j, label_j in test_data:

        img_j, label_j = img_j.to(device), label_j.to(device)

        with torch.autograd.no_grad():
            label_predicted = mlp.forward(img_j.view(-1, 28*28))
            total_test_loss  += criterion(torch.log(label_predicted), label_j.view(-1)).item()

    end_time = time.time()
    return total_test_loss, (end_time - start_time)


optimizer = optim.Adam(mlp.parameters(), lr=0.0001)
criterion = nn.NLLLoss()

for epoch in range(3):
    test_loss, response = run_epoch (mlp, train_data, test_data, optimizer, criterion)
    print('epoch ', epoch, ': ')
    print('\ttest_loss: ', test_loss)
    print('\tresponse(s): ', response)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./MNIST
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )